# from RDF dump, get ONS IRIs which don't exist in ONS

## do this for just one collection at a time
*this is just a little hack script*

In [41]:
import rdflib

# to-dos
    # !! speed this up !!
        # sort by vocab and only parse each vocab once for that group???
    # report on time taken?
    # add coll name (from dump file) to report top line and txt filename


In [42]:
# get all ONS.org values
filepath = input("Enter RDF dump filename with extension (must be in data_scratch/colls/)\n>>> ")

all_distinct_ons = []
g = rdflib.Graph().parse(f"../../data_scratch/colls/{filepath}")
result = g.query(
    """
    SELECT DISTINCT ?ons_resource
    WHERE {
        ?s ?p ?ons_resource .
        FILTER CONTAINS (str(?ons_resource), "http://opaquenamespace.org/ns/")
        }
    """)
counter = 0
for row in result:
    counter += 1
    # print(row.ons_resource)
    all_distinct_ons.append(str(row.ons_resource))
print(f"{counter} distinct ONS values found")

2720 distinct ONS values found


In [43]:
# iterate through these to see which fail when attempting to retrieve label
    # this is the part that I'd think needs to be sped up
    # by for example, parsing each ONS vocab only once for terms from that vocab
failures = []
for iri in all_distinct_ons:
    try:
        g = rdflib.Graph().parse(f"{iri}.nt")
    except:
        failures.append(iri)
print(f"Cannot retrieve a label for {len(failures)} ONS resources")

Cannot retrieve a label for 67 ONS resources


In [44]:
import json

coll_alias = filepath.split('/')[-1]
coll_alias = coll_alias.split('.')[0]

with open(f"ref_results/{coll_alias}_failures.json", 'w+') as jsonfile:
    json.dump({f"{coll_alias}_failures": failures}, jsonfile)


## put together a combined list from all collection results

In [59]:
import json

colls = ['angelus', 'building-or', 'chinavine', 'fealy', 'lowenstam', 
         'marketing-photos', 'nosatsu', 'uo-arch-photos']
counter = 0
combined_failures = []
for coll in colls:
    counter += 1
    with open(f"ref_results/{coll}_failures.json", "r") as jsonfile:
        faildict = json.load(jsonfile)
        faillist = faildict[f"{coll}_failures"]
        print(counter)
        # print(type(faillist))
        print(f"{len(faillist)} failed ONS IRIs in collection {coll}")
        if counter <= 1:
            for fail in faillist:
                combined_failures.append(fail)
        else:
            for fail in faillist:
                if fail not in combined_failures:
                    combined_failures.append(fail)
                else:
                    pass
with open("ref_results/combined_faillist.json", "w+") as jsonfile:
    json.dump({"combined_faillist": faillist}, jsonfile)

1
<class 'list'>
1 failed ONS IRIs in collection angelus
2
<class 'list'>
187 failed ONS IRIs in collection building-or
3
<class 'list'>
9 failed ONS IRIs in collection chinavine
4
<class 'list'>
0 failed ONS IRIs in collection fealy
5
<class 'list'>
22 failed ONS IRIs in collection lowenstam
6
<class 'list'>
2 failed ONS IRIs in collection marketing-photos
7
<class 'list'>
1 failed ONS IRIs in collection nosatsu
